In [44]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import math
import psycopg2
import datetime

today = datetime.date.today()
day = today.strftime('%d')
startMonth = today.strftime('%m')
endMonth = '0' + str(int(startMonth) + 1)
year = today.year

startDate = str(year)+'-'+str(startMonth)+'-'+'01'+' '+'00:00:00'
endDate = str(year)+'-'+str(endMonth)+'-'+'01'+' '+'00:00:00'

#Importing data
df = pd.read_csv('2018_hourly_data.csv')   #2018_hourly_data.csv #1year_hourly_data_2015.csv
# df.describe()

#Creating train and test set 
train=df[0:8784] #80% of data ( 0.8 * 8755 for training the model) 
test=df[8784:] # remaining 20% to test

#Aggregating the dataset at daily level
df.Timestamp = pd.to_datetime(df.dateformat_hour,format='%d-%m-%Y %H:%M') 
df.index = df.Timestamp 
df = df.resample('H').mean() #daily mean
train.Timestamp = pd.to_datetime(train.dateformat_hour,format='%d-%m-%Y %H:%M') 
train.index = train.Timestamp 
train = train.resample('H').mean() 
test.Timestamp = pd.to_datetime(test.dateformat_hour,format='%d-%m-%Y %H:%M') 
test.index = test.Timestamp 
test = test.resample('H').mean()

y_hat_avg = test.copy()
fit1 = sm.tsa.SARIMAX(train, order=(3, 1, 3),seasonal_order=(0,1,1,24)).fit()
y_hat_avg['SARIMA'] = predictions = fit1.predict(start=startDate, end=endDate, dynamic=True)

predictedValues = list()
for t in predictions:
    predictedValues.append(math.ceil(t))

print(predictedValues)




conn = psycopg2.connect("dbname=merakidb user=postgres password=postgres")
cur = conn.cursor()

yearParam = today.year
monthparam = today.month

x = datetime.datetime(yearParam,monthparam,1)
count = 0

cur.execute("DELETE FROM meraki.prediction_value_table")
for i in range(len(predictions)):
    
    if(count == 24):
        x += datetime.timedelta(days=1)
        count=0
    else:
        time = x.strftime("%x")
        visitorCount = predictedValues[i]
        cur.execute("INSERT INTO meraki.prediction_value_table (dateformat_date,count) VALUES (%s, %s);", (time, visitorCount))
        conn.commit()
        print(time,visitorCount)
    
    count += 1

C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\subhadip_jot\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_ret

[9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 123, 123, 124, 123, 123, 164, 163, 124, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 123, 123, 124, 123, 123, 164, 163, 124, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 123, 123, 124, 123, 123, 164, 164, 124, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 124, 123, 124, 123, 123, 164, 164, 124, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 124, 123, 124, 123, 123, 164, 164, 124, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 124, 123, 124, 123, 123, 164, 164, 124, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 124, 123, 124, 123, 123, 164, 164, 125, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 124, 123, 124, 123, 123, 164, 164, 125, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 124, 123, 124, 123, 123, 164, 164, 125, 124, 124, 9, 9, 9, 9, 9, 9, 9, 9, 9, 123, 124, 124, 125, 123, 124, 123, 124, 123, 123, 164, 16

05/24/19 125
05/24/19 10
05/25/19 10
05/25/19 10
05/25/19 10
05/25/19 10
05/25/19 10
05/25/19 10
05/25/19 10
05/25/19 124
05/25/19 125
05/25/19 125
05/25/19 126
05/25/19 124
05/25/19 124
05/25/19 124
05/25/19 125
05/25/19 124
05/25/19 124
05/25/19 165
05/25/19 164
05/25/19 125
05/25/19 125
05/25/19 125
05/25/19 10
05/26/19 10
05/26/19 10
05/26/19 10
05/26/19 10
05/26/19 10
05/26/19 10
05/26/19 10
05/26/19 124
05/26/19 125
05/26/19 125
05/26/19 126
05/26/19 124
05/26/19 124
05/26/19 124
05/26/19 125
05/26/19 124
05/26/19 124
05/26/19 165
05/26/19 165
05/26/19 125
05/26/19 125
05/26/19 125
05/26/19 10
05/27/19 10
05/27/19 10
05/27/19 10
05/27/19 10
05/27/19 10
05/27/19 10
05/27/19 10
05/27/19 124
05/27/19 125
05/27/19 125
05/27/19 126
05/27/19 124
05/27/19 125
05/27/19 124
05/27/19 125
05/27/19 124
05/27/19 124
05/27/19 165
05/27/19 165
05/27/19 125
05/27/19 125
05/27/19 125
05/27/19 10
05/28/19 10
05/28/19 10
05/28/19 10
05/28/19 10
05/28/19 10
05/28/19 10
05/28/19 10
05/28/19 124
05/28

In [8]:
/

05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 123
05/01/19 124
05/01/19 124
05/01/19 125
05/01/19 123
05/01/19 123
05/01/19 123
05/01/19 124
05/01/19 123
05/01/19 123
05/01/19 164
05/01/19 163
05/01/19 124
05/01/19 124
05/01/19 124
05/01/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 123
05/02/19 124
05/02/19 124
05/02/19 125
05/02/19 123
05/02/19 123
05/02/19 123
05/02/19 124
05/02/19 123
05/02/19 123
05/02/19 164
05/02/19 163
05/02/19 124
05/02/19 124
05/02/19 124
05/02/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 123
05/03/19 124
05/03/19 124
05/03/19 125
05/03/19 123
05/03/19 123
05/03/19 123
05/03/19 124
05/03/19 123
05/03/19 123
05/03/19 164
05/03/19 164
05/03/19 124
05/03/19 124
05/03/19 124
05/03/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 123
05/04/19 124
05/04/19 124
05/04/19 125
05/04/19 12

In [42]:
import datetime
today = datetime.date.today()
day = today.strftime('%d')
startMonth = today.strftime('%m')
endMonth = '0' + str(int(startMonth) + 1)
year = today.year

startDate = str(year)+'-'+str(startMonth)+'-'+'01'+' '+'00:00:00'
endDate = str(year)+'-'+str(endMonth)+'-'+'01'+' '+'00:00:00'

print(today.day)

28
